In [2]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [5]:
gid = 0
srv = 1
c = 0
cmd = {}
test_error = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    test_error[fold_num] = {}
    for lr in [0.01]:
        test_error[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/1/{}/{}/20000/0.0/".format(fold_num, lr)
            filename = "test-error-[\'{}\'].npy".format(o)

#             print(order)
            full_path = directory + filename
            my_file = Path(full_path)
    #         print(full_path)
            if not my_file.exists():
    #             print(fold_num, o)
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 1 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
                print(line)
    # #             cmd[c] = line
    #             with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
    #                 the_file.write("#!bin/bash \n")
    #                 the_file.write(line)
                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                test_error[fold_num][lr][order] = np.load(full_path).item()

print(c)                               

0


In [6]:
order

('mw', 'dw', 'dr', 'fridge', 'hvac')

In [8]:
results = {}
iters = {}
best_param = {}
for fold_num in range(5):
    results[fold_num] = {}
    best_param[fold_num] = {}
for fold_num in range(5):
    
    best_error = np.inf
    for lr in [0.01]:
        for order in test_error[fold_num][lr]:
            for iters in test_error[fold_num][lr][order]:
                cur_error = pd.Series(test_error[fold_num][lr][order][iters]).mean()
                if best_error > cur_error:
                    best_error = cur_error
                    best_param[fold_num]['iters'] = iters
                    best_param[fold_num]['order'] = order
                    best_param[fold_num]['lr'] = lr



In [9]:
best_param

{0: {'iters': 6000, 'lr': 0.01, 'order': ('dw', 'dr', 'hvac', 'fridge', 'mw')},
 1: {'iters': 20000,
  'lr': 0.01,
  'order': ('dr', 'fridge', 'mw', 'dw', 'hvac')},
 2: {'iters': 18000,
  'lr': 0.01,
  'order': ('dr', 'fridge', 'hvac', 'mw', 'dw')},
 3: {'iters': 19000,
  'lr': 0.01,
  'order': ('dw', 'fridge', 'mw', 'dr', 'hvac')},
 4: {'iters': 19000,
  'lr': 0.01,
  'order': ('fridge', 'dr', 'mw', 'dw', 'hvac')}}

In [10]:
np.save("../code/baseline/result/cnn-tree-param-1.npy", best_param)

In [30]:
best_error = {}
for fold_num in range(5):
    order = best_param[fold_num]['order']
    iters = best_param[fold_num]['iters']
    lr = best_param[fold_num]['lr']
    best_error[fold_num] = test_error[fold_num][lr][order][iters]

In [33]:
pd.DataFrame(best_error).T

,dr,dw,fridge,hvac,mw
0,54.142263,11.056612,26.687370,263.683233,6.811416
1,65.150710,16.477787,34.665792,245.316777,10.306474
2,71.015026,19.549901,26.368159,290.212051,6.388910
3,83.056380,20.592083,33.648809,338.435137,5.768193
4,63.915440,9.530771,33.535387,293.858105,13.075643


In [78]:
test_error[0]["dr', 'dw', 'fridge', 'hvac', 'mw"]

{1000: {'dr': 40.829784262484871,
  'dw': 11.024349682709262,
  'fridge': 29.872716202052374,
  'hvac': 187.02706319988212,
  'mw': 7.7511328845692731},
 2000: {'dr': 41.106744305441929,
  'dw': 11.024308222116865,
  'fridge': 30.699746206646108,
  'hvac': 196.96081333907924,
  'mw': 7.2772296308688302},
 3000: {'dr': 39.87791218008406,
  'dw': 11.024307820291078,
  'fridge': 30.953987375698659,
  'hvac': 198.19940872870089,
  'mw': 7.2221846988172826},
 4000: {'dr': 40.389561965165313,
  'dw': 11.024323599250435,
  'fridge': 30.797075706189002,
  'hvac': 197.12045763082634,
  'mw': 7.1889043471575578},
 5000: {'dr': 40.311422795187575,
  'dw': 11.0534663509653,
  'fridge': 30.659526995685923,
  'hvac': 194.44299547346819,
  'mw': 7.193766381530776},
 6000: {'dr': 40.254599978458124,
  'dw': 11.063868378625493,
  'fridge': 30.33197893184585,
  'hvac': 191.03339077991998,
  'mw': 7.1909012627725728},
 7000: {'dr': 39.773466390279616,
  'dw': 11.073374362652231,
  'fridge': 30.7200676030

In [55]:
for fold_num in range(5):
    best_param[i]
    best_order = pd.DataFrame(results[0]).mean().idxmin()

In [67]:
pd.DataFrame(results[0]).mean().min()

51.148836770916681

In [76]:
pd.Series(results[4][best_order])

dr         50.143053
dw          9.510564
fridge     36.966939
hvac      158.750494
mw         12.767054
dtype: float64